In [1]:
%pip install ezkl


  Using cached ezkl-12.0.1-cp37-abi3-macosx_10_12_x86_64.macosx_11_0_arm64.macosx_10_12_universal2.whl (22.8 MB)
Note: you may need to restart the kernel to use updated packages.


# steps that need to be run onece, when setting up the service

In [2]:
import ezkl
import os
import json

In [3]:
model_path = os.path.join('/Users/lnm/montreal/zk-hack/ezkl-example/models/ezkl-lenet.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [4]:
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs


In [11]:
#### generate dummy input data
import numpy as np
import json
dummy_input = np.random.rand(1, 1, 32, 32).astype(np.float32)

data = {'input_data': [dummy_input.reshape([-1]).tolist()]}
with open('input.json', 'w') as f:
    json.dump(data, f)

####

In [12]:

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
res = ezkl.calibrate_settings(model_path, settings_path, data_path)
#assert res == True


In [7]:

res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True


Future exception was never retrieved
future: <Future finished exception=RuntimeError('Failed to calibrate settings: [graph] [io] (/Users/lnm/montreal/zk-hack/ezkl-example/models/ezkl-lenet.onnx) stream did not contain valid UTF-8')>
RuntimeError: Failed to calibrate settings: [graph] [io] (/Users/lnm/montreal/zk-hack/ezkl-example/models/ezkl-lenet.onnx) stream did not contain valid UTF-8


In [13]:
# srs path
res = ezkl.get_srs("settings.json")


# generate proof: 

res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)

Future exception was never retrieved
future: <Future finished exception=RuntimeError('Failed to calibrate settings: [graph] [io] (/Users/lnm/montreal/zk-hack/ezkl-example/models/ezkl-lenet.onnx) stream did not contain valid UTF-8')>
RuntimeError: Failed to calibrate settings: [graph] [io] (/Users/lnm/montreal/zk-hack/ezkl-example/models/ezkl-lenet.onnx) stream did not contain valid UTF-8


# steps that need to be executed once per user

In [14]:
# "generate witness"
res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [16]:


proof_path = os.path.join('proof.json')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

{'instances': [['...']], 'proof': '0x288b68f630a7733fedcfed01a33b498efd1d3c57bc86b2a68e5a82fa6fb30136108f3bcdc322fb42fee336a7fc5819bbbc1eba72271648177ce5f74b40ab973222519b49ccee5b65272f89534e25507cf913f9acb70ed43df372c34c5fb9b44d14752d1ba2bcbca8004e80556c54c3f95d3134a8523f1f6382d09640f15d0b6f12b59274aba0e0bd42465ae8c7d77053d589f331b480e0220e357433089ea2a32d31ea4d673b87b08242b5a92d640a537e52edfc06810b62c25a808d853224cc1fe87a18216b23811b09dc265617704314baf78dc37cf478b4aa3ce96a0e036706f28e6452f3057a784f1deff30f076662931f7f6ed2fb5ea3ce12665bf47df71062d2776fea1b8e0f348bf3144e32947b67296b1aba3045b098410804e4086818db01dc10b3615ee7a5bb3e6e319f309aade01534e2268dd9a3d6a4492828bc22d4c92689abc64619f77666f45f37c2f8aaa14113f3f5cc60f32518bc25be4121a250dd71697060362e3125ff921f3da8fc69de527a324b36f58f347aa035d92b082e548953bf3a4a545f2943184a78d468c7ebc6d24ca38abb03e9f96d7db913ffff723b54437bc7b3a0303fa4bf6272adad8f0601e8c11b751f85ee642a610c5745156c7980a9d037ea3767024b66586b8ba1946ee72d545476dda0379bcd0ac

# Exporting the verifier for the EVM

In [17]:
ezkl.create_evm_verifier(
sol_code_path="code.sol", 
abi_path="abi", 
srs_path=None
)

<Future pending cb=[<builtins.PyDoneCallback object at 0x10cb5ba90>()]>